# API SNCF: Les retards de train

__Auteur :__ Steve Caron
__Date de creation :__ 28/01/2024

__Prerequis :__

* Python 3
* Une clef API SNCF stocké dans un environnement virtuel. Clé disponible sur : https://numerique.sncf.com/startup/api/
* Git

__Installation :__

* Cloner le projet

* Creer un environnement virtuel 

    ``py -m venv .venv``
    
* Activer l'environnement virtuel

    ``.venv/Scripts/activate``

* Installer les dépendance

    ``pip install -r requirements.txt``

__Params :__

* CLEF_API : nom sous lequel est enregistrer la cle API
* CODE_GARE : ID de la gare dont on veut récuperer les données

In [37]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json
import datetime

In [70]:
CLEF_API = "API_KEY"
CODE_GARE = "stop_area:SNCF:87686006"

Récupération de la clé API depuis l'environnement

In [71]:
load_dotenv()
api_key = os.getenv("API_KEY")

Je récupère la date du jour et à partir de celle-ci creer les arguments de ma requete. 

In [51]:
aujourdhui = datetime.date.today()
hier_debut_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=0, minute=0 ,second=0)
hier_fin_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=23, minute=59 ,second=59)

Creation des fonctions pour coder et decoder les datetime

In [56]:
def convertir_en_string(dt):
    '''Cette fonction convertit un datetime en chaine de caractère'''
    return datetime.datetime.strftime(dt,'%Y%m%dT%H%M%S')

In [64]:
def convertir_en_datetime(str):
    '''Cette fonction convertit une chaine de caractère en datetime'''
    return datetime.datetime.strptime((str[:8]+str[9:]),'%Y%m%d%H%M%S')

In [66]:
def requete_api(gare,date):

    base_url = "https://api.sncf.com/v1/coverage/sncf"
    physical_mode = "physical_mode:LongDistanceTrain"
    #Requete avec filtre sur les trains
    # requete =f"{base_url}/stop_areas/{gare}/physical_modes/{physical_mode}/departures?from_datetime={date}"
    #Requete sans le filtre sur les trains
    requete = f"{base_url}/stop_areas/{gare}/departures?from_datetime={date}"
    reponse = requests.get(requete, auth=(api_key,""))
    reponse_json = reponse.json()
    
    return reponse_json

In [67]:
def to_json(data,nom_fichier):
    '''Cette fonction permet d'enregistrer un fichier JSON'''
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [68]:
heure_min = convertir_en_string(hier_debut_journee)

In [72]:
reponse = requete_api(CODE_GARE,heure_min)
to_json(reponse,"test.json")